In [27]:
%reset -f


pip install pandas beancount matplotlib seaborn


In [28]:
from beancount.loader import load_file
import pandas as pd

entries, _, _ = load_file("./tab-data/ledger/dynamic.beancount")

txns = []
for entry in entries:
    if entry.__class__.__name__ == "Transaction":
        txn_id = id(entry)  # new unique ID for this transaction
        for posting in entry.postings:
            txns.append({
                "transaction_id": txn_id,
                "date": entry.date,
                "narration": entry.narration,
                "meta": entry.meta,
                "account": posting.account,
                "units": posting.units.number,
                "currency": posting.units.currency,
            })

df = pd.DataFrame(txns)


In [29]:
df

,transaction_id,date,narration,meta,account,units,currency
0,136912656992176,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Liabilities:Bar:Members:Bloemist,5.30,EUR
1,136912656992176,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Assets:InitialBalances,-5.30,EUR
2,136912657006064,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Liabilities:Bar:Members:Koen,-0.50,EUR
3,136912657006064,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Assets:InitialBalances,0.50,EUR
4,136912657004272,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Liabilities:Bar:Members:Bart,-11.80,EUR
...,...,...,...,...,...,...,...
100120,136912751826128,2025-05-29,Gust gave DJefke a gift of €41.70,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Liabilities:Bar:Members:DJefke,-41.70,EUR
100121,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Assets:Inventory:Bar,-1,SODA
100122,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Liabilities:Bar:Members:Mateo,1,SODA
100123,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,{'filename': '/mnt/c/Repos/home/hackerspace/ba...,Liabilities:Bar:Members:Mateo,1.50,EUR


# Examining columns

In [30]:
for column in df.columns:
    try:
        unique_count = df[column].nunique()
    except TypeError:
        unique_count = df[column].apply(lambda x: str(x)).nunique()
    print(f"{column}: {unique_count} unique values")


transaction_id: 24761 unique values
date: 2050 unique values
narration: 9383 unique values
meta: 24761 unique values
account: 90 unique values
units: 847 unique values
currency: 81 unique values


# Get rid of meta column

In [31]:
# Show first row
first_row = df.iloc[1000]
# print(first_row)

# Show shortened meta (first 100 chars)
meta_short = str(first_row['meta'])
print(meta_short)


{'filename': '/mnt/c/Repos/home/hackerspace/bar-graphs/tab-data/ledger/imported-from-spacebar.beancount', 'lineno': 1713, '__tolerances__': {'EUR': Decimal('0.005')}}


From the meta we ONLY want the type of the purchase

In [32]:
df['type'] = df['meta'].apply(lambda m: m.get('type') if isinstance(m, dict) else None)
df = df.drop(columns=['meta'])
# Show results
df['type'].unique()



array([None, 'purchase', 'deposit', 'transfer'], dtype=object)

In [33]:
df

,transaction_id,date,narration,account,units,currency,type
0,136912656992176,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Liabilities:Bar:Members:Bloemist,5.30,EUR,None
1,136912656992176,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Assets:InitialBalances,-5.30,EUR,None
2,136912657006064,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Liabilities:Bar:Members:Koen,-0.50,EUR,None
3,136912657006064,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Assets:InitialBalances,0.50,EUR,None
4,136912657004272,1970-01-01,Initial balance transfer for Liabilities:Bar:M...,Liabilities:Bar:Members:Bart,-11.80,EUR,None
...,...,...,...,...,...,...,...
100120,136912751826128,2025-05-29,Gust gave DJefke a gift of €41.70,Liabilities:Bar:Members:DJefke,-41.70,EUR,transfer
100121,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,Assets:Inventory:Bar,-1,SODA,purchase
100122,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,Liabilities:Bar:Members:Mateo,1,SODA,purchase
100123,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,Liabilities:Bar:Members:Mateo,1.50,EUR,purchase


# Examining transactions

Each transaction is a collection of postings.

In [34]:
postings_per_transaction = df.groupby("transaction_id").size().reset_index(name="num_postings")

postings_per_transaction = postings_per_transaction.sort_values(
    by='num_postings',
    ascending=False
).reset_index(drop=True)

top_id = postings_per_transaction.iloc[0]["transaction_id"]

postings_per_transaction


,transaction_id,num_postings
0,136912891273936,17
1,136912788706624,12
2,136912855782272,12
3,136912867002288,12
4,136912704356160,12
...,...,...
24756,136912875339840,2
24757,136912875353728,2
24758,136912875348464,2
24759,136912875501296,2


Let's look at all postings in this transaction

In [35]:
df[df["transaction_id"] == top_id]


,transaction_id,date,narration,account,units,currency,type
88846,136912891273936,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-1,COCKTAIL,purchase
88847,136912891273936,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,1,COCKTAIL,purchase
88848,136912891273936,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-3,GERMAN,purchase
88849,136912891273936,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,3,GERMAN,purchase
88850,136912891273936,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-6,DUVEL,purchase
88851,136912891273936,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,6,DUVEL,purchase
88852,136912891273936,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-8,CHIP,purchase
88853,136912891273936,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,8,CHIP,purchase
88854,136912891273936,2023-08-03,Els bought 30 items for €118.00,Assets:Inventory:Bar,-8,WATER,purchase
88855,136912891273936,2023-08-03,Els bought 30 items for €118.00,Liabilities:Bar:Members:Els,8,WATER,purchase


# Get products

In [36]:
import yaml

# Load YAML and convert to DataFrame
with open("./tab-data/static/products.yml", "r") as f:
    price_data = yaml.safe_load(f)
df_products = pd.DataFrame(price_data)

# Expand the 'payback' dictionary into its own columns
if 'payback' in df_products.columns:
    payback_df = df_products['payback'].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series)
    payback_df.columns = [f'payback_{col}' for col in payback_df.columns]
    df_products = pd.concat([df_products.drop(columns=['payback']), payback_df], axis=1)

df_products

,name,price,event_price,currency,category,visible,payback_account,payback_amount
0,Soda,1.5,2.50,SODA,drink,True,NaN,NaN
1,Cecemel,0.7,1.00,CHOCO_MILK,drink,False,NaN,NaN
2,Fritz / Club Mate Cola,2.5,3.50,GERMAN,drink,True,NaN,NaN
3,Club Mate,3.0,4.00,CM,drink,True,NaN,NaN
4,Tea,0.5,0.75,TEA,drink,False,NaN,NaN
...,...,...,...,...,...,...,...,...
63,Pizza,2.5,4.00,PIZZA,food,False,Liabilities:Bar:Members:Yvan,1.5
64,SPRITE,2.0,3.00,SPRITE,DRINK,False,NaN,NaN
65,Cha-Cha,0.5,0.75,CHACHA,food,False,NaN,NaN
66,Leffe,2.0,3.00,LEFFE,alcohol,False,NaN,NaN


See all products where a payback is present. This means a member should be paid back every time an items is purchased of this kind.

In [37]:
df_products[df_products['payback_account'].notna()]

,name,price,event_price,currency,category,visible,payback_account,payback_amount
11,Wristband,1.0,1.5,WRISTBAND,swag,True,Liabilities:Bar:Members:Marquis,0.50
14,Passport,2.5,3.6,PASSPORT,swag,True,Liabilities:Bar:Members:Bart,1.20
15,Postcard,2.5,3.5,POSTCARD,NaN,NaN,Liabilities:Bar:Members:NewlineJar,1.00
59,ROM,1.0,1.5,ROM,food,False,Liabilities:Bar:Members:Thequux,0.50
62,Ice Cream,1.0,1.5,ICE_CREAM,food,False,Liabilities:Bar:Members:Yvan,0.63
63,Pizza,2.5,4.0,PIZZA,food,False,Liabilities:Bar:Members:Yvan,1.50


# Get recent values

In [38]:
df["date"] = pd.to_datetime(df["date"])  # Convert first
cutoff = pd.Timestamp.now() - pd.DateOffset(years=3)
recent_df = df[df["date"] >= cutoff]    # Then filter

In [39]:
recent_df

,transaction_id,date,narration,account,units,currency,type
69743,136912886701904,2022-06-02,Nachtvlinder bought 2 items for €2.60,Assets:Inventory:Bar,-1,BOUNTY,purchase
69744,136912886701904,2022-06-02,Nachtvlinder bought 2 items for €2.60,Liabilities:Bar:Members:Maja,1,BOUNTY,purchase
69745,136912886701904,2022-06-02,Nachtvlinder bought 2 items for €2.60,Assets:Inventory:Bar,-1,FRITZ,purchase
69746,136912886701904,2022-06-02,Nachtvlinder bought 2 items for €2.60,Liabilities:Bar:Members:Maja,1,FRITZ,purchase
69747,136912886701904,2022-06-02,Nachtvlinder bought 2 items for €2.60,Liabilities:Bar:Members:Maja,2.60,EUR,purchase
...,...,...,...,...,...,...,...
100120,136912751826128,2025-05-29,Gust gave DJefke a gift of €41.70,Liabilities:Bar:Members:DJefke,-41.70,EUR,transfer
100121,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,Assets:Inventory:Bar,-1,SODA,purchase
100122,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,Liabilities:Bar:Members:Mateo,1,SODA,purchase
100123,136912751826912,2025-05-29,Mateo bought 1 items for €1.50,Liabilities:Bar:Members:Mateo,1.50,EUR,purchase
